In [1]:
import numpy as np
import tensorflow as tf
import os
#from riptide.models.cifar_resnet import cifar_resnet20_v1
from riptide.binary.q_cifar_resnet import cifar_resnet20_v1
from riptide.binary.HWGQ_funcs import Quantize, HWGQuantize, load_clusters, load_bits
from riptide.binary.HWGQ_layers import Config
from slim.preprocessing.cifarnet_preprocessing import preprocess_image

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train
y_train = y_train.astype(np.int32)
x_test = x_test
y_test = y_test.astype(np.int32)

In [3]:
def train_transform(data, label):
    #data = tf.image.convert_image_dtype(data, tf.float32)
    #data = tf.random_crop(data, [32,32,3])
    #data = tf.image.random_flip_left_right(data)
    #data = tf.image.random_brightness(data, 0.1)
    #data = tf.image.random_contrast(data, 0.9, 1.1)
    #data = tf.image.random_saturation(data, 0.9, 1.1)
    #data = tf.image.per_image_standardization(data)
    data = preprocess_image(data, 32, 32, is_training=True)
    return data, label

def test_transform(data, label):
    #data = tf.image.convert_image_dtype(data, tf.float32)
    #data = tf.image.resize_images(data, [32,32])
    #data = tf.image.per_image_standardization(data)
    data = preprocess_image(data, 32, 32, is_training=False)
    return data, label

def train_input_fn():
    ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    ds = ds.map(train_transform)
    ds = ds.batch(100)
    ds = ds.shuffle(1000)
    ds = ds.repeat(120)
    return ds

def eval_input_fn():
    ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    ds = ds.map(test_transform)
    ds = ds.batch(100)
    ds = ds.repeat(1)
    return ds

In [4]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    clusters = load_clusters(2)
    actQ = HWGQuantize
    weightQ = Quantize
    config = Config(actQ=actQ, weightQ=weightQ, clusters=clusters)
    with config:
        model = cifar_resnet20_v1()
    #model = cifar_resnet20_v1()
    logits = model(features)
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        lr = tf.train.exponential_decay(0.1, tf.train.get_global_step(), 20000, 0.1)
        optimizer = tf.train.MomentumOptimizer(learning_rate = lr, momentum=0.9)
        #optimizer = tf.train.AdamOptimizer(learning_rate = lr)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [5]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/cifar_convnet_model16")
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=None)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff62419a750>, '_model_dir': '/tmp/cifar_convnet_model16', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


In [6]:
tf.estimator.train_and_evaluate(mnist_classifier, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/cifar_convnet_model16/model.ckpt.
INFO:tensorflow:loss = 2.3277047, step = 0
INFO:tensorflow:global_step/sec: 17.6089
INFO:tensorflow:loss = 2.1212425, step = 100 (5.682 sec)
INFO:tensorflow:global_step/sec: 31.4075
INFO:tensorflow:loss = 2.0012314, step = 200 (3.184 sec)
INFO:tensorflow:global_step/sec: 31.6007
INFO:tensorflow:loss = 1.8509387, step = 300 (3.164 sec)
INFO:tensorflow:global_step/sec: 31.4302
INFO:t

({'accuracy': 0.7789, 'global_step': 60000, 'loss': 0.683983}, [])

In [7]:
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-22-19:35:28
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/cifar_convnet_model15/model.ckpt-60000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-22-19:35:31
INFO:tensorflow:Saving dict for global step 60000: accuracy = 0.8692, global_step = 60000, loss = 0.54936296
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 60000: /tmp/cifar_convnet_model15/model.ckpt-60000
{'loss': 0.54936296, 'global_step': 60000, 'accuracy': 0.8692}
